In [20]:
# This file is part of Grafite <https://github.com/marcocosta97/grafite>.
# Copyright (C) 2023 Marco Costa.
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib
import matplotlib.pyplot as plt
import itertools, os
from pathlib import Path

In [21]:
width = 5.47807
width_small = 3.50069
max_height = 7.944

rc_fonts = {
    "font.family": "serif",
    "font.size": 10,
    "text.usetex": True,
    "text.latex.preamble": r"\usepackage[tt=false,type1=true]{libertine}\usepackage[varqu]{zi4}\usepackage[libertine]{newtxmath}"}
matplotlib.rcParams.update(rc_fonts)

out_folder = Path('./figures')
if not out_folder.exists():
    out_folder.mkdir()
base_csv_path = Path('./results')
if not out_folder.exists():
    raise FileNotFoundError(
        "error, cannot find the results folder")

range_filters = ['Grafite', 'Bucketing', 'SNARF', 'SuRF', 'Proteus', 'Rosetta', 'REncoder', 'REncoder_SS', 'REncoder_SE']
range_filters_style_kwargs = {'Grafite': {'marker': 'o', 'color': 'dimgray', 'zorder': 10, 'label': 'Grafite'},
                              'SNARF': {'marker': '^', 'color': 'C1', 'label': 'SNARF'},
                              'SuRF': {'marker': 's', 'color': 'C2', 'label': 'SuRF'},
                              'Proteus': {'marker': 'X', 'color': 'C3', 'label': 'Proteus'},
                              'Rosetta': {'marker': 'd', 'color': 'C4', 'label': 'Rosetta'},
                              'REncoder': {'marker': '>', 'color': 'C5', 'label': 'REncoder'},
                              'REncoder_SS': {'marker': 'v', 'color': 'C6', 'label': 'REncoderSS'},
                              'REncoder_SE': {'marker': 'h', 'color': 'C8', 'label': 'REncoderSE'},
                              'Bucketing': {'marker': '*', 'color': 'C0', 'zorder': 9, 'label': 'Bucketing'}}

range_filters_cmaps = {'Grafite': {'cmap': cm.Greys}, 
                        'SNARF': {'cmap': cm.Oranges},
                        'SuRF': {'cmap': cm.Greens},
                        'Proteus': {'cmap': cm.Reds},
                        'Rosetta': {'cmap': cm.Purples}}

empty_markers_style = {'linestyle': ':', 'fillstyle': 'none', 'alpha': 0.6, 'markersize': 4}
lines_style = {'markersize': 4, 'linewidth': 0.7, 'fillstyle': 'none'}


keys_synth = ['kuniform']
query_synth = ['qcorrelated', 'quniform']

labels_name = {'kuniform': r'$\textsc{Uniform}$', 
               'knormal': r'$\textsc{Normal}$',
               'qcorrelated': r'$\textsc{Correlated}$', 
               'quniform': r'$\textsc{Uncorrelated}$',
               'books': r'$\textsc{Books}$',
               'osm': r'$\textsc{Osm}$',
               'fb': r'$\textsc{Fb}$'}

keys_real = ['books', 'osm', 'fb']
query_range = [0, 5, 10]
query_range_label = [f"Point queries", "Small range", "Large range", "Mixed range"]

datasets_synth = list(itertools.product(keys_synth, query_synth))
datasets_real = keys_real

fpr_test_path = f'{base_csv_path}/fpr_test'
def get_file(filter, range_size, dataset_name, query_name = "", path=fpr_test_path):
    filter = filter.lower()
    if dataset_name in keys_synth or query_name:
        p = Path(f'{path}/{dataset_name}/{range_size}_{query_name}/{filter}.csv')
    else:
        p = Path(f'{path}/{dataset_name}/{range_size}/{filter}.csv')
    if not p.exists:
        raise FileNotFoundError(f'error, {p} does not exist')
    return p

title_font_size = 10
legend_font_size = 7.5
ylabel_font_size = 10
xlabel_font_size = 10

max_x_axis_bpk = 30

## Figure 4-5: bpk vs fpr for heuristics and bounded range filter

In [22]:
sorted_dirs = sorted(os.listdir(fpr_test_path), reverse=True)
if len(sorted_dirs) < 1:
    raise FileNotFoundError(
        "error, cannot find the latest test executed")

fpr_test_path = Path(fpr_test_path + '/' + sorted_dirs[0])

In [23]:
fpr_real_test_path = f'{base_csv_path}/fpr_real_test'

sorted_dirs = sorted(os.listdir(fpr_real_test_path), reverse=True)
if len(sorted_dirs) < 1:
    raise FileNotFoundError(
        "error, cannot find the latest test executed")

fpr_real_test_path = Path(fpr_real_test_path + '/' + sorted_dirs[0])

In [24]:

def print_fpr_test(filters, workloads, name):
    nrows = len(workloads)
    ncols = len(query_range)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey='row', figsize=(width * 0.8, max_height/1.4))

    iterate = list(itertools.product(workloads, filters, enumerate(query_range)))

    for (x, ds, r) in iterate:
        row = workloads.index(x)
        (idx, ran) = r
        try:
            if type(x) is tuple:
                data = pd.read_csv(get_file(ds, r[1], x[0], x[1], path=fpr_test_path))
            else:
                data = pd.read_csv(get_file(ds, r[1], x, path=fpr_real_test_path))
        except: continue
        data['fpr_opt'] = data['false_positives'] / data['n_queries']
        data.plot("bpk", "fpr_opt", ax=axes[row][idx], **range_filters_style_kwargs[ds], **lines_style)

    ticks = [1, 1e-01, 1e-02, 1e-03, 1e-04, 1e-05, 1e-06, 0]

    for ax in axes.flatten():
        ax.set_yscale('symlog', linthresh=(1e-06))
        ax.set_xlim(right=max_x_axis_bpk)
        ax.set_yticks(ticks)
        ax.set_ylim(bottom=-0.0000003, top=1.9)
        ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(2))
        ax.set_xlabel('Space [bits/key]', fontsize=xlabel_font_size)
        ax.get_legend().remove()
        ax.autoscale_view()
        ax.margins(0.04)
    
    for ax in axes:
        ax[0].yaxis.set_minor_locator(matplotlib.ticker.LogLocator(numticks=10, subs="auto"))
        
    for i, k in list(enumerate(workloads)):
        if type(k) is tuple:
            axis_title = f'{labels_name[k[1]]}'
        else:
            axis_title = f'{labels_name[k]}'
        axes[i][0].set_ylabel(axis_title + "\nFalse Positive Rate", fontsize=ylabel_font_size)
        
    for i, _ in list(enumerate(query_range)):
        axes[0][i].set_title(query_range_label[i], fontsize=title_font_size)

    fig.subplots_adjust(wspace=0.1)
    fig.savefig(f'{out_folder}/fpr_test_{name}_nolegend.pdf', bbox_inches='tight', pad_inches=0.01)
    lines, labels = axes[0][1].get_legend_handles_labels()

    if len(filters) > 4:
        ncol = len(filters) // 2
        bbox = (0.5, 1.65)
    else:
        ncol = len(filters)
        bbox = (0.5, 1.5)
        
    axes[0][1].legend(lines, labels, loc='upper center', bbox_to_anchor=bbox,
            fancybox=True, shadow=False, ncol=ncol, fontsize=legend_font_size)
    fig.savefig(f'{out_folder}/fpr_test_{name}.pdf', bbox_inches='tight', pad_inches=0.01)

In [25]:
import collections

def generate_tables(filters, workloads):
    nrows = len(workloads)
    workload_row = [collections.defaultdict(list) for _ in range(nrows)]
    iterate = list(itertools.product(workloads, filters, enumerate(query_range)))

    for (x, ds, r) in iterate:
        row = workloads.index(x)
        try:
            if type(x) is tuple:
                data = pd.read_csv(get_file(ds, r[1], x[0], x[1], path=fpr_test_path))
            else:
                data = pd.read_csv(get_file(ds, r[1], x, path=fpr_real_test_path))
        except: continue
        data["single_query_time"] = (data["query_time"] / data["n_queries"]) * 10**6
        workload_row[row][range_filters_style_kwargs[ds]['label']].append(round(data["single_query_time"].mean(), 2))

    mean_row = [collections.defaultdict(list) for _ in range(nrows)]
    for i in range(nrows):
        for key, value in workload_row[i].items():
            mean_row[i][key].append(round(np.mean(value)))
        default_value = mean_row[i][filters[0]][0]
        for key, value in mean_row[i].items():
            mean_row[i][key].append(round(value[0]/default_value, 2))

    df_list = []
    for i in range(nrows): 
        df = pd.DataFrame()
        for key, value in mean_row[i].items():
            col_name = 'Avg ns/query'
            df.at[key, 'avg'] = value[0]
            df.at[key, col_name] = str(value[0]) + ' (' + str(value[1]) + '$\\times$)'

        # sort by 'temp' column ignoring the row of index 'Grafite'
        df.iat[0, df.columns.get_loc('avg')] = -1
        df = df.sort_values(by=['avg'])
        # remove the 'temp' column
        df = df.drop('avg', axis=1)
        df_list.append(df)
        
    return df_list

def print_table(df_list, filename, space_em_list):
    filled_sections = []
    filled_sections.append("\\begin{tabular}{rl}")
    for i, df in enumerate(df_list):
        filled_sections.append("\\toprule")
        if i == 0:
            filled_sections.append("Range filter & Avg ns/query \\\\")
            filled_sections.append("\\midrule")
        table_string = df.to_latex(index=True, header=False, escape=False)
        filled_sections.extend(table_string.split("\n")[3:-2])    
        filled_sections.append("\\vspace{" + space_em_list[i] + "}\\\\")
    
    filled_sections.append("\\end{tabular}")
    # Join all filled sections
    filled_template = "\n".join(filled_sections)
    
    with open(filename, "w") as file:
        file.write(filled_template)   

In [26]:
workloads = [('kuniform', 'qcorrelated'), ('kuniform', 'quniform'), ('books'), ('osm')]
filters_heuristics = ['Bucketing', 'SuRF', 'Proteus', 'SNARF', 'REncoder_SS', 'REncoder_SE']

In [ ]:
print_fpr_test(filters_heuristics, workloads, 'heuristics')

In [ ]:
df_list = generate_tables(filters_heuristics, workloads)
spaces_list = ['.7em', '.7em', '.7em', '1.4em']
print_table(df_list, f'{out_folder}/table_heuristics.tex', spaces_list)

In [29]:
filters_bounded = ['Grafite', 'Rosetta', 'REncoder']

In [ ]:
print_fpr_test(filters_bounded, workloads, 'bounded')

In [ ]:
df_list = generate_tables(filters_bounded, workloads)
spaces_list = ['4em', '4.4em', '4.4em', '3em']
print_table(df_list, f'{out_folder}/table_bounded.tex', spaces_list)

## Figure 7: input size vs construction time

In [32]:
size_test_path = f'{base_csv_path}/constr_time_test'

sorted_dirs = sorted(os.listdir(size_test_path), reverse=True)
if len(sorted_dirs) < 1:
    raise FileNotFoundError(
        "error, cannot find the latest test executed")

size_test_path = Path(size_test_path + '/' + sorted_dirs[0])

keys_size = [5, 6, 7, 8]
labels_keys_size = [f'$10^{x}$' for x in keys_size]

matplotlib.rcParams['hatch.linewidth'] = 0.1

In [ ]:
# Figure 7: input size vs construction time
old_range_filters = range_filters
range_filters = ['Grafite', 'Bucketing', 'SNARF', 'SuRF', 'Proteus', 'Rosetta', 'REncoder']

fig, ax = plt.subplots(figsize=(width_small * 0.78, 0.4 * width))

width_bars = 0.12  # the width of the bars
multiplier = 0

iterate = list(itertools.product(keys_size, range_filters))
for (r, ds) in iterate:
    i = r - min(keys_size)
    try:
        data = pd.read_csv(get_file(ds, 5, f'kuniform_{r}', query_name='quniform', path=size_test_path))
    except: continue
    if data.empty or data["build_time"].empty: continue

    if "modelling_time" in data.columns:
        build_time = np.mean(data['build_time'])/data['n_keys'] * 10**6
        modelling_time = np.mean(data['modelling_time'])/data['n_keys'] * 10**6
        ax.bar(range_filters.index(ds) * width_bars + i, build_time, width_bars, color=range_filters_style_kwargs[ds]['color'])
        ax.bar(range_filters.index(ds) * width_bars + i, modelling_time, width_bars, label='_nolegend_', bottom=build_time, color=range_filters_style_kwargs[ds]['color'], alpha=0.1, hatch='//////')
    else:
        build_time = np.mean(data['build_time'])/data['n_keys'] * 10**6
        ax.bar(range_filters.index(ds) * width_bars + i, build_time, width_bars, color=range_filters_style_kwargs[ds]['color'])

ax.set_ylabel('Construction Time [ns/key]', fontsize=ylabel_font_size)
ax.legend(range_filters, loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=False, ncol=1, fontsize=legend_font_size)
ax.set_xticks(np.arange(len(keys_size)) + 3 * width_bars, labels_keys_size)
ax.set_xlabel('Number of Keys', fontsize=xlabel_font_size)
ax.yaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(100))
leg = ax.get_legend()
for ds in range_filters:
    leg.legend_handles[range_filters.index(ds)].set_color(range_filters_style_kwargs[ds]['color'])


fig.savefig(f'{out_folder}/constr_time_test.pdf', bbox_inches='tight', pad_inches=0.01)
range_filters = old_range_filters

## Figure 6: bpk vs query time (true queries)

In [34]:
true_test_path = f'{base_csv_path}/true_test'

sorted_dirs = sorted(os.listdir(true_test_path), reverse=True)
if len(sorted_dirs) < 1:
    raise FileNotFoundError(
        "error, cannot find the latest test executed")

true_test_path = Path(true_test_path + '/' + sorted_dirs[0])

In [ ]:
# Figure 6: bpk vs query time (true queries)

keys_synth = ['kuniform']
nrows = len(keys_synth)
ncols = len(query_range)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey='row', figsize=(width * 0.8, 0.2 * width))

iterate = list(itertools.product(range_filters, enumerate(query_range)))

for (ds, r) in iterate:
    (idx, ran) = r
    try:
        data = pd.read_csv(get_file(ds, r[1], 'kuniform', 'qtrue', true_test_path))
    except: continue
    data["single_query_time"] = (data["query_time"] / data["n_queries"]) * 10**6
    data.plot("bpk", "single_query_time", ax=axes[idx], **range_filters_style_kwargs[ds], **lines_style)

ticks = [10**2, 10**3, 10**4, 10**5]

for ax in axes.flatten():
    ax.set_xlim(right=max_x_axis_bpk)
    ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(2))
    ax.set_xlabel('Space [bits/key]', fontsize=xlabel_font_size)
    ax.set_yscale('log')
    ax.get_legend().remove()
    ax.yaxis.set_minor_locator(matplotlib.ticker.LogLocator(numticks=10, subs='auto'))
    ax.set_yticks(ticks)

for i, k in list(enumerate(keys_synth)):
    axis_title = f'{k}'
    axes[i].set_ylabel("Time [ns/query]", fontsize=ylabel_font_size)
    
for i, _ in list(enumerate(query_range)):
    axes[i].set_title(query_range_label[i], fontsize=title_font_size)
    
fig.subplots_adjust(wspace=0.1)
fig.savefig(f'{out_folder}/true_queries_test_nolegend.pdf', bbox_inches='tight', pad_inches=0.01)
box = axes[2].get_position()
lines, labels = axes[0].get_legend_handles_labels()
# order = [0,1,5,2,6,3,7,4,8]
axes[2].legend(lines, labels, 
                  loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=False, ncol=2, fontsize=legend_font_size, columnspacing=0.5)

fig.savefig(f'{out_folder}/true_queries_test.pdf', bbox_inches='tight', pad_inches=0.01)

## Figure 3: Correlation

In [36]:
corr_test_path = f'{base_csv_path}/corr_test'

sorted_dirs = sorted(os.listdir(corr_test_path), reverse=True)
if len(sorted_dirs) < 1:
    raise FileNotFoundError(
        "error, cannot find the latest test executed")

corr_test_path = Path(corr_test_path + '/' + sorted_dirs[0])

corr_degrees = range(0, 11)

In [ ]:
# Figure 3: Correlation
fig, axes = plt.subplots(2, 3, sharex=True, sharey='row', figsize=(width, 0.6 * width))

range_filters_corr = ['Grafite', 'Bucketing', 'SNARF', 'SuRF', 'Proteus', 'Rosetta', 'REncoder', 'REncoder_SS', 'REncoder_SE']

iterate = list(itertools.product(range_filters_corr, enumerate(query_range), corr_degrees))

values = [dict() for x in range(len(query_range))]
time_values = [dict() for x in range(len(query_range))]

for ds in range_filters_corr:
    for r in range(len(query_range)):
        values[r][ds] = []
        time_values[r][ds] = []
        
xlabels_corr = [ x/10 for x in corr_degrees ]

for (ds, r, deg) in iterate:
    (idx, ran) = r
    try:
        data = pd.read_csv(get_file(ds, r[1], f'kuniform_{deg}', 'qcorrelated', corr_test_path))
        data['fpr_opt'] = data['false_positives'] / data['n_queries']  
        time = data['query_time'][0]/data['n_queries'][0] * 10**6
        fpr = data['fpr_opt'][0]
    except:
        fpr = None
        time = None
    values[idx][ds].append(fpr)
    time_values[idx][ds].append(time)
    
for r in range(len(query_range)):
    for key, data_list in values[r].items():
        axes[0][r].plot(xlabels_corr, data_list, **range_filters_style_kwargs[key], **lines_style)
            
for r in range(len(query_range)):
    for key, data_list in time_values[r].items():
        axes[1][r].plot(xlabels_corr, data_list, **range_filters_style_kwargs[key], **lines_style)   
    axes[1][r].set_ylim(10**1, 10**6)
    axes[1][r].set_yscale('log')
    
axes[1][0].set_ylabel('Time [ns/query]', fontsize=ylabel_font_size)

for ax in axes.flatten():
    ax.margins(0.04)
    ax.set_yscale('symlog', linthresh=(1e-05))
    ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(0.1))
    
for ax in axes[1].flatten():
    ax.set_xlabel('Correlation Degree', fontsize=xlabel_font_size)
    ax.yaxis.set_minor_locator(matplotlib.ticker.LogLocator(numticks=10, subs='auto'))
    
for i, _ in list(enumerate(query_range)):
    axes[0][i].set_title(query_range_label[i], fontsize=xlabel_font_size)
plt.subplots_adjust(hspace=0.1, wspace=0.15)
axes[0][0].set_ylabel('False Positive Rate', fontsize=ylabel_font_size)
axes[0][0].yaxis.set_minor_locator(matplotlib.ticker.LogLocator(numticks=10, subs='auto'))

plt.savefig(f'{out_folder}/corr_test_twolines_nolegend.pdf', bbox_inches='tight', pad_inches=0.01)
box = axes[0][2].get_position()
lines, labels = axes[0][0].get_legend_handles_labels()
# order = [0,4,1,5,2,6,3,4]
order = list(range(len(range_filters_corr)))
axes[0][2].legend([lines[idx] for idx in order],[labels[idx] for idx in order], 
                  loc='center left', bbox_to_anchor=(1, -0.05),
                  fancybox=True, shadow=False, ncol=1, fontsize=legend_font_size)

plt.savefig(f'{out_folder}/corr_test_twolines.pdf', bbox_inches='tight', pad_inches=0.01)

## Figure 1: Correlation (small)

In [ ]:
# Figure 1: Correlation (small)
range_filters_corr = ['Grafite', 'SNARF', 'SuRF', 'Proteus', 'Rosetta', 'REncoder']


fig, axes = plt.subplots(1, 2, sharex=True, figsize=(width_small * 1.2, 0.3 * width))
query_range_small = [5]

iterate = list(itertools.product(range_filters_corr, enumerate(query_range_small), corr_degrees))
values = [dict() for x in range(len(query_range_small))]
time_values = [dict() for x in range(len(query_range_small))]

for ds in range_filters_corr:
    for r in range(len(query_range_small)):
        values[r][ds] = []
        time_values[r][ds] = []
        
xlabels_corr = [ x/10 for x in corr_degrees ]

for (ds, r, deg) in iterate:
    (idx, ran) = r
    try:
        data = pd.read_csv(get_file(ds, r[1], f'kuniform_{deg}', 'qcorrelated', corr_test_path))
        data['fpr_opt'] = data['false_positives'] / data['n_queries']
        fpr = data['fpr'][0]
        time = data['query_time'][0]/data['n_queries'][0] * 10**6
    except:
        fpr = None
        time = None
    values[idx][ds].append(fpr)
    time_values[idx][ds].append(time)
    
for key, data_list in values[0].items():
    axes[0].plot(xlabels_corr, data_list, **range_filters_style_kwargs[key], **lines_style)
            
for key, data_list in time_values[0].items():
    axes[1].plot(xlabels_corr, data_list, **range_filters_style_kwargs[key], **lines_style)   
axes[1].set_yscale('log')

axes[1].set_ylabel('Time [ns/query]', fontsize=ylabel_font_size)
axes[1].set_ylim(10**2, 10**4)

for ax in axes.flatten():
    ax.set_yscale('log')
    ax.set_xlabel('Correlation Degree', fontsize=xlabel_font_size)
    ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(0.1))
    ax.set_xticks([0, 0.5, 1])

box = axes[1].get_position()
lines, labels = axes[0].get_legend_handles_labels()
lines2, labels2 = axes[1].get_legend_handles_labels()
axes[1].xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
axes[1].legend(lines, labels, loc='upper center', bbox_to_anchor=(-0.3, -0.35),
          fancybox=True, shadow=False, ncol=3, columnspacing=0.5, fontsize=legend_font_size)
axes[0].set_ylabel('False Positive Rate', fontsize=ylabel_font_size)
axes[0].yaxis.set_minor_locator(matplotlib.ticker.LogLocator(numticks=10, subs='auto'))
axes[0].set_yticks([1, 1e-01, 1e-02, 1e-03, 1e-04])
fig.align_ylabels([axes[0], axes[1]])

plt.subplots_adjust(wspace=0.5)
plt.savefig(f'{out_folder}/corr_test_small.pdf', bbox_inches='tight', pad_inches=0.01)